In [1]:
from tqdm import tqdm
import numpy as np
import torchaudio
import torch
import os
import wespeaker
import pandas as pd
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import os
import subprocess
import sys

In [2]:
os.chdir("..")

In [3]:
from src.utils import *

In [4]:
test_dir = os.path.join("data", "vox1_test_wav")
df = scan_directory_voxceleb1(test_dir)

In [5]:
df

,path,person_id,utterance_env,utterance_filename
0,data/vox1_test_wav/id10295/nt7dNRvlEHE/00005.wav,id10295,nt7dNRvlEHE,00005.wav
1,data/vox1_test_wav/id10295/nt7dNRvlEHE/00004.wav,id10295,nt7dNRvlEHE,00004.wav
2,data/vox1_test_wav/id10295/nt7dNRvlEHE/00001.wav,id10295,nt7dNRvlEHE,00001.wav
3,data/vox1_test_wav/id10295/nt7dNRvlEHE/00003.wav,id10295,nt7dNRvlEHE,00003.wav
4,data/vox1_test_wav/id10295/nt7dNRvlEHE/00002.wav,id10295,nt7dNRvlEHE,00002.wav
...,...,...,...,...
4869,data/vox1_test_wav/id10305/vbDKF8bZJQA/00004.wav,id10305,vbDKF8bZJQA,00004.wav
4870,data/vox1_test_wav/id10305/vbDKF8bZJQA/00010.wav,id10305,vbDKF8bZJQA,00010.wav
4871,data/vox1_test_wav/id10305/vbDKF8bZJQA/00001.wav,id10305,vbDKF8bZJQA,00001.wav
4872,data/vox1_test_wav/id10305/vbDKF8bZJQA/00003.wav,id10305,vbDKF8bZJQA,00003.wav


In [11]:
from pathlib import Path
from typing import Union
from src.utilities.prep_audio import get_audio_tensor

# for each row in df, extract audio tensor
output_dir= os.path.join("data", "vox1_test_segments")
os.makedirs(output_dir, exist_ok=True)
for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    try:
        audio_tensor = get_audio_tensor(row["path"])
    except Exception as e:
        print(f"Error reading file {row['path']}: {e}")
        continue
    for i, segment in enumerate(audio_tensor):
        segment_name = "_seg_" + str(i)
        os.makedirs(output_dir + "/" + row["person_id"], exist_ok=True)
        segment_path = output_dir + "/" + row["person_id"] + "/" + row["utterance_filename"].replace(".wav","") + segment_name + ".wav"
        torchaudio.save(segment_path, segment, 16000)

100%|██████████| 4874/4874 [00:08<00:00, 566.40it/s]


In [13]:
# check that the segments were created 

from src.utils import scan_directory_voxceleb2


segments = scan_directory_voxceleb2(output_dir)

In [14]:
segments

,path,person_id,utterance_filename
0,data/vox1_test_segments/id10295/00004_seg_8.wav,id10295,00004_seg_8.wav
1,data/vox1_test_segments/id10295/00007_seg_0.wav,id10295,00007_seg_0.wav
2,data/vox1_test_segments/id10295/00013_seg_0.wav,id10295,00013_seg_0.wav
3,data/vox1_test_segments/id10295/00001_seg_7.wav,id10295,00001_seg_7.wav
4,data/vox1_test_segments/id10295/00001_seg_6.wav,id10295,00001_seg_6.wav
...,...,...,...
9114,data/vox1_test_segments/id10305/00016_seg_2.wav,id10305,00016_seg_2.wav
9115,data/vox1_test_segments/id10305/00018_seg_9.wav,id10305,00018_seg_9.wav
9116,data/vox1_test_segments/id10305/00009_seg_6.wav,id10305,00009_seg_6.wav
9117,data/vox1_test_segments/id10305/00007_seg_11.wav,id10305,00007_seg_11.wav


In [16]:
# Add a column with duration in samples
durations = []

for index, row in tqdm(segments.iterrows(), total=segments.shape[0]):
    try:
        info = torchaudio.info(row["path"])
        durations.append(info.num_frames)
    except Exception as e:
        print(f"Error reading file {row['path']}: {e}")
        durations.append(None)

segments["duration_samples"] = durations
unique_durations = segments["duration_samples"].unique()
print(unique_durations)

100%|██████████| 9119/9119 [00:00<00:00, 14941.52it/s]

[64000]
